# Convolutional Neural Network Model for Detecting Spoofed Aircraft
### Purpose: This notebook creates a Convolutional Neural Network capable of detecting spoofed aircraft. It was trained using a training data set of pre-generated spoofed aircraft alongside a dataset of valid ADS-B records. It outputs a binary classification (spoofed or not) for a plane alongside confidence level. 

In [1]:
import json
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Valid data parsing first
with open('./samples/valid/aircraft_valid.json') as json_file:
    data_valid = json.load(json_file)
    
# Parse spoofed data
with open('./samples/spoofed/aircraft_spoofed.json') as json_file:
    data_spoofed = json.load(json_file)
 
aircraft_data_valid = data_valid['aircraft']
aircraft_data_spoofed = data_spoofed['aircraft']

print("Datasets imported!")

2023-11-24 12:15:48.172556: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Datasets imported!


# Process JSON Files Containing Data

# Prepare Dataset to be Placed into Neural Network
### Split into Training and Testing Sets

In [16]:
# Define a function to preprocess the data
def preprocess_data(data):
    features = []
    labels = []

    for aircraft in data:
        try:
            feature_vector = [
                handle_alt_baro(aircraft.get('alt_baro', 0)),
                float(aircraft.get('gs', 0)),
                float(aircraft.get('track', 0)),
                float(aircraft.get('baro_rate', 0)),
                float(aircraft.get('lat', 0)),
                float(aircraft.get('lon', 0)),
                float(aircraft.get('seen_pos', 0)),
                float(aircraft.get('messages', 0)),
                float(aircraft.get('seen', 0)),
                float(aircraft.get('rssi', 0)),
            ]
            features.append(feature_vector)
            labels.append(1 if aircraft.get('is_spoofed', False) else 0)
        except ValueError as e:
            # Handle the case where a non-numeric value is encountered
            print(f"Ignoring data point due to non-numeric value: {aircraft}, Field {e}")

    return np.array(features), np.array(labels)

def handle_alt_baro(alt_baro):
    if alt_baro == "ground":
        return 0.0
    else:
        return alt_baro
    
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(10,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
spoofed_features, spoofed_labels = preprocess_data(aircraft_data_spoofed)
not_spoofed_features, not_spoofed_labels = preprocess_data(aircraft_data_valid)

# Combine the data
all_features = np.vstack([spoofed_features, not_spoofed_features])
all_labels = np.concatenate([spoofed_labels, not_spoofed_labels])

print(all_features, all_labels)

[[ 1.728100e+04  8.700000e+02  1.880000e+02 ...  9.970000e+02
   1.790000e+02 -1.100000e+02]
 [ 1.981200e+04  1.610000e+02  7.800000e+01 ...  2.500000e+01
   5.130000e+02 -2.060000e+02]
 [ 2.134300e+04  6.660000e+02  8.700000e+01 ...  8.920000e+02
   4.380000e+02 -2.460000e+02]
 ...
 [ 0.000000e+00  0.000000e+00  0.000000e+00 ...  1.237000e+03
   2.700000e+01 -2.720000e+01]
 [ 0.000000e+00  0.000000e+00  0.000000e+00 ...  3.617607e+06
   2.190000e+01 -1.290000e+01]
 [ 0.000000e+00  0.000000e+00  0.000000e+00 ...  6.101024e+06
   5.300000e+00 -4.300000e+00]] [1 1 1 ... 0 0 0]


In [18]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(all_features, all_labels, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

Epoch 1/10
169/169 [==============================] - 1s 3ms/step - loss: 41277.1992 - accuracy: 0.9337 - val_loss: 0.4110 - val_accuracy: 0.9970
Epoch 2/10
169/169 [==============================] - 0s 2ms/step - loss: 2.8839 - accuracy: 0.9972 - val_loss: 0.4435 - val_accuracy: 0.9970
Epoch 3/10
169/169 [==============================] - 0s 2ms/step - loss: 1.8332 - accuracy: 0.9978 - val_loss: 0.3570 - val_accuracy: 0.9970
Epoch 4/10
169/169 [==============================] - 0s 2ms/step - loss: 1.0547 - accuracy: 0.9974 - val_loss: 0.1169 - val_accuracy: 0.9985
Epoch 5/10
169/169 [==============================] - 0s 2ms/step - loss: 1.2598 - accuracy: 0.9981 - val_loss: 1.1113 - val_accuracy: 0.9963
Epoch 6/10
169/169 [==============================] - 0s 2ms/step - loss: 0.2978 - accuracy: 0.9983 - val_loss: 2.8358 - val_accuracy: 0.9963
Epoch 7/10
169/169 [==============================] - 0s 2ms/step - loss: 2.2605 - accuracy: 0.9981 - val_loss: 0.0055 - val_accuracy: 0.9970
Ep

In [19]:
model_name = "Spoof_Detection.h5"
model.save(model_name)
print(f"Model saved to {model_name}!")

/usr/local/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved to Spoof_Detection.h5!


# References and Resources
#### [Reference dump1090 README](https://github.com/SDRplay/dump1090/blob/master/README-json.md)
#### [Data Samples from ADSB Exchange](https://www.adsbexchange.com/data-samples/)
#### [2022 Data Sample from ADSB Exchange](https://samples.adsbexchange.com/readsb-hist/2022/05/01/)
#### [ADSB Exchange API](https://www.adsbexchange.com/version-2-api-wip/)
#### [Raspberry Pi & FlightAware Setup](https://elmwoodelectronics.ca/blogs/news/tracking-and-logging-flights-with-ads-b-flight-aware-and-raspberry-pi)
#### [IEEE Research on ADS-B Signals](https://ieeexplore.ieee.org/document/9377975)